# Particle Swarm Optimization

Sources:
- http://www.swarmintelligence.org/tutorials.php
- https://github.com/tisimst/pyswarm/blob/master/pyswarm/pso.py

At iteration $k$, particle $i$ has posistion $x^i_k$ defined recursively as:

$$x^i_{k+1} = x^i_k + v^i_{k+1}$$

where $v$ is its velocity defined as follows:

$$ v^i_{k+1} = w_k \cdot v^i_{k} + c_1 r_1 (p^i_k - x^i_k) + c_2 r_2 (p^g_k - x^i_k)$$

Where:

- $p^i_k$ is the particle's best known position
- $p^g_k$ is the global (all particles) best known position
- $w_k$ is an inertia parameter
- $c_1$ is the cognitive parameter
- $c_2$ is the social parameter
- $r_1, r_2$ are random numbers in $(0, 1)$

In plain English, the velocity update is defined as:

new_velocity = (inertia * previous_velocity) + (move in the direction of the personal best) + (move in the direction of the global best)

## Code

In [12]:
# import two useful libraries/modules
import numpy as np
import random
import multiprocessing
from multiprocessing import Pool
from multiprocessing.pool import ThreadPool

#--------------------------------------------------------------------------------------------------------
class Particle:
    """A Python Class for a simple particle."""
    
    def __init__(self, upper, lower, ndim,
                 c1 = 1, c2 = 2, w = .5):
        '''Initiate a Particle with an upper and lower bound, and a number of dimensions.'''
        # Initiate position and velocity randomly
        self.position = np.array([random.uniform(lower, upper) for _ in range(ndim)])
        self.velocity = np.array([random.uniform(-1, 1) for _ in range(ndim)])
        # These attributes are here to store the "memory" of the function
        self.personal_best_position = self.position # initial position
        self.personal_best_fitness = np.inf # infinity
        self.c1 = c1
        self.c2 = c2
        self.w = w
    
    
    def __str__(self):
        '''what will be called when you print a Particle instance, could be useful for debugging.'''
        return("I am a particle with best position {},\ncurrent position {},\nand velocity {}.\n"
               .format(self.personal_best_position, self.position, self.velocity))
    
    
    def update_velocity(self, global_best):
        '''update a particle's velocity'''
        # two random coefficients in uniform([0, 1])
        r1 = random.random()
        r2 = random.random()
        self.velocity = (self.w*self.velocity + # inertia
                         self.c1*r1*(self.personal_best_position - self.position) + # cognitive term
                         self.c2*r2*(global_best - self.position)) # social term
        
            
    def move(self):
        '''Moves a Particle at a new iteration. Deals with out-of-bound cases (We assume they are ok.)'''
        self.position = self.position + self.velocity
        # need to deal with when the particle goes out of bound? Not here.

# ----------------------------------------------------------------------------------------------------------------
class PSO:
    """
    +----------------------------------------------------------+
    + Parallel Solver instance for Particle Swarm Optimizer.   +
    + (Basically the mastermind which commands the Particles.) +
    +----------------------------------------------------------+
    
    Instantiated with:
    -----------------
    - num_particles: how many particles to use?
    
    - function: function to minimize. IT MUST BE of the form:
                function np.array[coordinates] --- f() ---> [list] (or np.array)
                typically, the function returns a list comprehension.
                
    - n_iter: number of iterations to be performed. To be replaced/completed with convergence criterions.
    
    - ndim: dimensionality of the search space.
    
    - lower, upper: lower and upper bounds for the search space
    (for now, it's a box i.e. they are the same accross all dimensions.)
    
    - c1, c2, w: cognitive, social, and inertia parameters. To be tuned to the specific problem.
    
    - parallel: whether to evaluate the fitness of particles in parallel. `False` by default as a speed-boost is
    unlikely in most simple settings.
    
    - epsilon: defines convergence. If an update to the global best is smaller than epsilon, then the algorithm
               has converged.
    
    """
    
    def __init__(self, num_particles, function, n_iter, ndim, lower = -10, upper = 10,
                 c1 = 1, c2 = 2, w = .5, parallel = False, threadpool = True, epsilon = 10e-7):
        '''Initiate the solver'''
        # create all the Particles, stored in a list.
        self.particles = [Particle(lower, upper, ndim, c1, c2, w) for _ in range(num_particles)]
        self.fitnesses = np.array([])
        # store global best and associated fitness
        self.global_best = np.array([])
        self.global_best_fitness = np.inf # infinity
        self.function = function # function to be optimised
        self.n_iter = n_iter # num of iterations
        self.parallel = parallel
        self.epsilon = epsilon
        self.hasConverged = False
        if parallel:
            if threadpool:
                self.pooler = ThreadPool(3)
            else:
                self.pooler = Pool(multiprocessing.cpu_count() - 1)
            
    def get_fitnesses(self):
        """Evaluate all fitnesses (in parallel is self.parallel is True.)"""
        if self.parallel:
            fitnesses = self.pooler.apply(self.function, [[part.position for part in self.particles]])
            self.fitnesses = np.array(fitnesses)
            
        else :
            fitnesses = self.function([part.position for part in self.particles])
            self.fitnesses = np.array(fitnesses)
    
    def update_particles(self):
        '''update particle best known personal position'''
        for i in range(len(self.fitnesses)):
            if self.fitnesses[i] < self.particles[i].personal_best_fitness:
                self.particles[i].personal_best_fitness = self.fitnesses[i]
                self.particles[i].personal_best_position = self.particles[i].position          
        
    def update_best(self):
        '''Find the new best global position and update it.
        Additionally check for convergence.'''
        if np.any(self.fitnesses < self.global_best_fitness):
            fit_before = self.global_best_fitness
            self.global_best_fitness = np.min(self.fitnesses)
            self.global_best = self.particles[np.argmin(self.fitnesses)].position
            
            #check convergence
            if abs(fit_before - self.global_best_fitness) < self.epsilon:
                self.hasConverged = True
                      
    def move_particles(self):
        '''Run one iteration of the algorithm. Update particles velocity and move them.'''
        for particle in self.particles:
            particle.update_velocity(self.global_best)
            particle.move()
    
    def __str__(self):
        '''Print best global position when calling `print(pso instance)`'''
        return """Current best position: {}
        With fitness: {}""".format(self.global_best, self.global_best_fitness)
    
    def run(self, verbose = True):
        '''Run the algorithm and print the result. By default, update us every 50 iterations.'''
        
        if verbose:
            print("Running the PSO algorithm in parallel with {} particles, for {} iterations.\n".
                  format(len(self.particles), self.n_iter))
        
        for iteration in range(self.n_iter):
            # this happens in parallel (synchronous only rn)
            self.get_fitnesses()
            # this doesn't
            self.update_particles()
            self.update_best()
            self.move_particles()
            
            if (iteration % 50 == 0) & verbose==True:
                print("Iteration number " + str(iteration))
                print("Current best fitness: " + str(self.global_best_fitness))
                print("\n")
                
            if self.hasConverged == True:
                break
                
        if verbose:
            print("Found minimum at {} with value {}.".format(self.global_best, self.global_best_fitness))
            
        return(self.global_best)

## Tests and benchmarks

In [13]:
# This function is cheap to evaluate
def quad_function(li):
    return [(x[0] + 2*x[1] - 3)**2 + (x[0] - 2)**2 for x in li]

In [57]:
%%time
pso = PSO(2000, high_dim_rosenbrock, 100, 30, epsilon=10e-7, c1 = 1, c2 = .5, w = 1)
a = pso.run(verbose=False)

CPU times: user 19.5 s, sys: 57.6 ms, total: 19.6 s
Wall time: 19.6 s


In [58]:
print(a)
high_dim_rosenbrock([a])

[ 0.7327308   0.81936219  0.81525751  0.62618025  0.43977101  0.08992285
 -0.26254927  0.28720321  0.10317264  0.166867   -0.13201519 -0.18996056
  0.2925446  -0.42161255  0.54084625  0.58278775  0.34615907  0.18203068
  0.01672118  0.17591301 -0.47833256  0.6361995   0.70743684  0.96170033
  0.99362301  1.09940952  1.20885627  1.22693857  1.35781105  2.01436491]


[191.62786975922438]

In [59]:
%%time
pso = PSO(2000, high_dim_rosenbrock, 100, 30, epsilon=10e-7, c1 = 1, c2 = .5, w = 1, parallel=True)
b = pso.run(verbose = False)

CPU times: user 4.84 s, sys: 197 ms, total: 5.04 s
Wall time: 30.6 s


In [60]:
%%time
pso = PSO(2000, high_dim_rosenbrock, 100, 30, epsilon=10e-7, c1 = 1, c2 = .5, w = 1, parallel=True, threadpool=True)
s = pso.run(verbose = False)

CPU times: user 19.2 s, sys: 29.4 ms, total: 19.2 s
Wall time: 19.3 s


In [67]:
s

array([ 0.5693535 ,  0.19695028, -0.01649498, -0.04482905, -0.22147895,
        0.01998439,  0.17128307, -0.17823111,  0.14654732,  0.12068108,
        0.12262196, -0.08824805, -0.050288  , -0.035337  ,  0.19914663,
        0.05089694, -0.14477562, -0.0034723 , -0.14874531,  0.07998543,
        0.22727497,  0.10904129, -0.05186458, -0.00808873, -0.10373874,
       -0.00400616,  0.02206105,  0.27174969,  0.78844727,  0.79671553])

In [33]:
for p in pso.particles:
    print(p)

I am a particle with best position [-0.89331655  0.31884318 -0.62630102 -0.74515617 -0.14559912 -1.45895656
 -0.37354061 -0.90815858 -0.56862616  1.46786354  0.98300583  0.58972006
 -0.03922431 -0.30193074  1.21015538  0.50565977  1.31854441  0.98470791
  1.24715843 -0.39333225 -1.34937537  1.60555967  1.52462781  1.11918287
 -0.38940177  1.00683069 -0.88254228  0.90433822 -0.44181731 -0.10005169],
current position [-0.89774974  0.31707276 -0.61874018 -0.74641091 -0.15824991 -1.45327522
 -0.36251207 -0.89786326 -0.57607557  1.46869961  0.99545602  0.60038468
 -0.03506142 -0.30280371  1.22358858  0.5026902   1.31866161  0.97811333
  1.2526019  -0.39164471 -1.35537393  1.60993522  1.5290185   1.12881166
 -0.40840083  0.99851611 -0.87963177  0.8967224  -0.44684163 -0.0973582 ],
and velocity [-0.00443319 -0.00177041  0.00756083 -0.00125474 -0.01265079  0.00568134
  0.01102854  0.01029532 -0.00744941  0.00083607  0.01245019  0.01066463
  0.00416289 -0.00087298  0.0134332  -0.00296957  0.000

In [32]:
high_dim_rosenbrock([s])

[3886.443374772515]

The [Rosenbrock Function](https://en.wikipedia.org/wiki/Rosenbrock_function)

$f(x,y)=(1-x)^{2}+100(y-x^{2})^{2}$

In [17]:
def rosenbrock(li):
    def f(x):
        return (1-x[0])**2 + 100*(x[1] - x[0]**2)**2
    return [f(x) for x in li]

In [5]:
pso = PSO(20, rosenbrock, 100, ndim = 2)
pso.run()

Running the PSO algorithm in parallel with 20 particles, for 100 iterations.

Iteration number 0
Current best fitness: 132.43694620701896


Found minimum at [1.00038968 1.00080949] with value 2.4174906789499275e-07.


array([1.00038968, 1.00080949])

High dimensional Rosenbrock

${\displaystyle f(\mathbf {x} )=\sum _{i=1}^{N-1}[100(x_{i+1}-x_{i}^{2})^{2}+(1-x_{i})^{2}]\quad {\mbox{where}}\quad \mathbf {x} =[x_{1},\ldots ,x_{N}]\in \mathbb {R} ^{N}.}$

In [22]:
def high_dim_rosenbrock(li):
    def f(x):
        return sum([(1-x[i])**2 + 100*(x[i+1] - x[i]**2)**2 for i in range(len(x) - 1)])
    return [f(x) for x in li]

In [ ]:
high_dim_rosenbrock([np.array([0, 0, 0]), np.array([1, 1, 1])])

In [ ]:
pso = PSO(100, high_dim_rosenbrock, 100, ndim = 5)
sol = pso.run()

## Griewank function

$$f(x) = 1+{\frac  {1}{4000}}\sum _{{i=1}}^{n}x_{i}^{2}-\prod _{{i=1}}^{n}\cos \left({\frac  {x_{i}}{{\sqrt  {i}}}}\right)$$

In [ ]:
def griewank(li):
    def f(x):
        return 1 + (1/4000) * sum([xi**2 for xi in x]) - np.product([np.cos(x[i]/np.sqrt(i+1)) for i in range(len(x))])
    return [f(x) for x in li]

In [ ]:
pso = PSO(500, griewank, 100, ndim = 3, parallel=False, c1 = 3, c2 = .5)
pso.run()

For this function, hyperparameter tuning matters a lot!
(here, high c1 to let particles explore more!)

## Schaffer's F6

$$f(x)=0.5+\frac{sin^2(\sqrt{x_1^2 + x_2^2})-0.5}{[1+0.001 \cdot (x_1^2 + x_2^2)]^2}$$

In [ ]:
def schaffer_f6(li):
    def f(x):
        return .5 + ((np.sin(np.sqrt(x[0]**2 + x[1]**2))**2) - .5)/((1 + 0.001*(x[0]**2 + x[1]**2))**2)
    return [f(x) for x in li]

In [ ]:
schaffer_f6([[0, 0], [1, 0], [1, 1]])

In [ ]:
pso = PSO(100, schaffer_f6, 100, 2)
pso.run()

## Rastrigin function

$$f(\mathbf {x} )=An+\sum _{i=1}^{n}\left[x_{i}^{2}-A\cos(2\pi x_{i})\right]$$

where $ A=10 $ and $x_{i}\in [-5.12,5.12]$.

In [ ]:
def rastrigin(li):
    def f(x):
        return 10*len(x) + sum([xi**2 - 10 * np.cos(2*np.pi*xi) for xi in x])
    return [f(x) for x in li]

rastrigin([[0, 0, 0, 0, 0], [1, 1], [1, 2, 3, 4]])

# Solver is in the `PSPSO.py` module !

In [2]:
from PSPSO import Particle, PSO
from functions import quad_function

# Instantiate PSO solver
pso = PSO(num_particles = 20, function = quad_function, n_iter = 100, ndim = 2)

pso.run()

Running the PSO algorithm in parallel with 20 particles, for 100 iterations.

Iteration number 0
Current best error: 18.166881777526612


Iteration number 50
Current best error: 3.727553918035805e-12


Found minimum at [2.  0.5] with value 8.0667327796468e-23.


array([2. , 0.5])

In [ ]:
pso_par = PSO(10000, rastrigin, 10, ndim = 30,
          lower = -5.12, upper = 5.12, c1 = 2, c2 = .33,
          parallel=True)

In [ ]:
%%time
pso_par.run(verbose = True)

In [ ]:
pso_nonpar = PSO(10000, rastrigin, 10, ndim = 30,
          lower = -5.12, upper = 5.12, c1 = 2, c2 = .33,
          parallel=False)

In [ ]:
pso.parallel

In [ ]:
%%time
pso.run()